1. write requests interaction
1. write response method for each trail type
    1. list all trail types
    1. write a single method for each trail type
    1. write a main method that calls the other methods
1.  write a agent method that calls the main method
    1. define an agent state
    2. response methods might mutate the agent state
    3. define logic that defines how social learning influences the agent state
1.  make a for loop that iterates over multiple agents


In [24]:


# @session_router.get('/{experiment_type}/{prolific_id}', response_model_by_alias=False)
# async def get_current_trial(experiment_type: str, prolific_id: str) -> Union[Trial, SessionError]:
#     """
#     Get current trial from the session.
#     """
#     print("get_current_trial", flush=True)
#     # find session and trial for the subject
#     session = await get_session(prolific_id, experiment_type)

#     # return error if session is not available
#     if isinstance(session, SessionError):
#         return session

#     trial = await prepare_trial(session)

#     await session.save()
#     print("trial: ", trial, flush=True)
#     return trial

# @session_router.post('/{prolific_id}/{trial_id}')
# async def post_current_trial_results(
#         prolific_id: str,
#         trial_id: int,
#         body: Union[Solution, WrittenStrategy, Advisor, PostSurvey, None
#         ] = None) -> Union[TrialSaved, SessionError, TrialError]:
#     # find session assigned to the subject
#     session = await get_session(prolific_id)

#     # return error if session is not available
#     if isinstance(session, SessionError):
#         return session

#     # get current trial
#     trial = session.trials[session.current_trial_num]

#     # check if trial type is correct
#     if trial.id != trial_id:
#         return TrialError(message='Trial type is not correct')

#     await save_trial(body, session, trial, trial.trial_type)

#     await update_session(session)

#     return TrialSaved()



In [25]:
import requests
import random
import json
import uuid
from common.models.trial import Solution, Advisor, AdvisorSelection, WrittenStrategy, PostSurvey, Trial, TrialSaved, TrialError, SessionError

In [26]:
# create new experiment

baseurl = "http://0.0.0.0:5050"

url = f"{baseurl}/admin/config"

experiment_type = 'test2'

payload = json.dumps({
  "active": True,
  "created_at": "2023-10-17T09:57:36.204000",
  "redirect_url": "https://app.prolific.co/submissions/complete",
  "experiment_type": experiment_type,
  "rewrite_previous_data": True,
  "seed": 1,
  "n_generations": 3,
  "n_ai_players": 1,
  "networks_path": "data/23_11_30",
  "n_sessions_per_generation": 4,
  "n_advise_per_session": 2,
  "n_session_tree_replications": 1,
  "conditions": [
    "w_ai",
    "wo_ai"
  ],
  "n_social_learning_blocks": 1,
  "n_social_learning_networks_per_block": 2,
  "n_practice_trials": 1,
  "n_demonstration_trials": 3,
  "simulate_humans": False,
  "social_learning_trials": [
    "observation",
    "repeat",
    "try_yourself"
  ],
  "main_only": True
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic YWRtaW46YWRtaW4='
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"id":"65ad9c1df1d736d11ac84535","active":true,"created_at":"2024-01-21T22:35:09.875418","redirect_url":"https://app.prolific.co/submissions/complete","experiment_type":"test2","rewrite_previous_data":true,"networks_path":"data/23_11_30","seed":1,"n_generations":3,"n_ai_players":1,"n_sessions_per_generation":4,"n_advise_per_session":2,"n_session_tree_replications":1,"conditions":["w_ai","wo_ai"],"n_social_learning_blocks":1,"n_social_learning_networks_per_block":2,"n_practice_trials":1,"n_demonstration_trials":3,"simulate_humans":false,"social_learning_trials":["observation","repeat","try_yourself"],"main_only":true}


In [27]:
def get_trial(prolific_id, experiment_type):
    url = f"{baseurl}/session/{experiment_type}/{prolific_id}"
    payload={}
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    try:
      return Trial(**response.json())
    except:
      return None

def post_trial(prolific_id, trial_id, body):
    url = f"{baseurl}/session/{prolific_id}/{trial_id}"
    payload=body
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.status_code == 200

In [28]:
from pathlib import Path
from common.utils.utils import estimate_solution_score, estimate_average_player_score
from common.models.network import Network


networks_path = Path("../data/23_11_30")
network_data = json.load(open(networks_path / "networks.json"))
solutions_myopic = json.load(open(networks_path / "solution__myopic.json"))
solutions_m1 = json.load(open(networks_path / "solutions" / "0.json"))
networks_by_id = {n["network_id"]: n for n in network_data}
solutions_myopic_by_id = {s["network_id"]: s for s in solutions_myopic}
solutions_m1_by_id = {s["network_id"]: s for s in solutions_m1}


def get_solution(network_id, solution_type):
    network = networks_by_id[network_id]
    # get the solution for the network
    if solution_type == "myopic":
        solution = solutions_myopic_by_id[network_id]
    elif solution_type == "machine":
        solution = solutions_m1_by_id[network_id]
    else:
        raise ValueError("Invalid solution type")

    solution['moves'][0] = network['starting_node']
    score = estimate_solution_score(Network(**network), solution['moves'], 10)
    assert score > 0, f"Invalid solution score: {score}"

    return Solution(**solution)

In [29]:


def handle_instruction_trial(trial, state):
    body = None
    return body, state

def handle_individual_trial(trial, state):
    solution = get_solution(trial.network.network_id, 'myopic')
    return solution.json(), state

def handle_written_strategy_trial(trial, state):
    strategy = WrittenStrategy(
        strategy=''
    )
    body = strategy.json()
    return body, state

def handle_demonstration_trial(trial, state):
    solution = get_solution(trial.network.network_id, 'myopic')
    score = estimate_solution_score(trial.network, solution.moves, 10)
    assert score > 0, f"Invalid solution score: {score}"
    return solution.json(), state

def handle_debriefing_trial(trial, state):
    body = None
    return body, state

def handle_social_learning_selection_trial(trial, state):
    advisor_selection = trial.advisor_selection
    advisor = advisor_selection.advisor_ids
    scores = advisor_selection.scores
    advisor = random.choice(advisor)
    selection = Advisor(
        advisor_id=advisor
    )
    body = selection.json()
    return body, state

def handle_observation_trial(trial, state):
    body = None
    return body, state

def handle_repeat_trial(trial, state):
    solution = trial.advisor.solution
    body = solution.json()
    return body, state

def handle_try_yourself_trial(trial, state):
    solution = get_solution(trial.network.network_id, 'myopic')
    return solution.json(), state


def handle_trial(trial, state):
    if trial.trial_type == "instruction":
        return handle_instruction_trial(trial, state)
    elif trial.trial_type == "individual":
        return handle_individual_trial(trial, state)
    elif trial.trial_type == "written_strategy":
        return handle_written_strategy_trial(trial, state)
    elif trial.trial_type == "demonstration":
        return handle_demonstration_trial(trial, state)
    elif trial.trial_type == "debriefing":
        return handle_debriefing_trial(trial, state)
    elif trial.trial_type == "social_learning_selection":
        return handle_social_learning_selection_trial(trial, state)
    elif trial.trial_type == "observation":
        return handle_observation_trial(trial, state)
    elif trial.trial_type == "repeat":
        return handle_repeat_trial(trial, state)
    elif trial.trial_type == "try_yourself":
        return handle_try_yourself_trial(trial, state)
    else:
        raise ValueError(f"{trial.trial_type} is an invalid trial type")

In [30]:
trials = []

finished = False
while not finished:
    prolific_id = uuid.uuid4().hex[:8]
    state = {}
    current_trial_id = None
    while True:
        trail = get_trial(prolific_id, experiment_type)
        if trail is None:
            finished = True
            break
        print(f'ProlificId: {prolific_id}, TrialId: {trail.id}, TrialType: {trail.trial_type}')
        if trail.id == current_trial_id:
            break
        current_trial_id = trail.id
        body, state = handle_trial(trail, state)
        post_trial(prolific_id, trail.id, body)
        trial_clean = json.loads(trail.json())
        # session_clean = json.loads(trail.session.json())
        trials.append({'trial': trial_clean, 'prolific_id': prolific_id})



# save trials as json
import json
with open('trials.json', 'w') as f:
    json.dump(trials, f, indent=4)

ProlificId: 06569c5a, TrialId: 0, TrialType: instruction


ProlificId: 06569c5a, TrialId: 1, TrialType: individual
ProlificId: 06569c5a, TrialId: 2, TrialType: individual
ProlificId: 06569c5a, TrialId: 3, TrialType: written_strategy
ProlificId: 06569c5a, TrialId: 4, TrialType: instruction
ProlificId: 06569c5a, TrialId: 5, TrialType: demonstration
ProlificId: 06569c5a, TrialId: 6, TrialType: demonstration
ProlificId: 06569c5a, TrialId: 7, TrialType: demonstration
ProlificId: 06569c5a, TrialId: 8, TrialType: debriefing
ProlificId: 06569c5a, TrialId: 8, TrialType: debriefing
ProlificId: 65da5846, TrialId: 0, TrialType: instruction
ProlificId: 65da5846, TrialId: 1, TrialType: individual
ProlificId: 65da5846, TrialId: 2, TrialType: individual
ProlificId: 65da5846, TrialId: 3, TrialType: written_strategy
ProlificId: 65da5846, TrialId: 4, TrialType: instruction
ProlificId: 65da5846, TrialId: 5, TrialType: demonstration
ProlificId: 65da5846, TrialId: 6, TrialType: demonstration
ProlificId: 65da5846, TrialId: 7, TrialType: demonstration
ProlificId: 65d